In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# # /content/drive/MyDrive/Proj_WT/DataSets/박정열/덕수궁_가을.csv
# import os
# from google.colab import drive
# # 구글 드라이브 접근
# ROOT = "/content/drive"
# drive.mount(ROOT)
# from os.path import join
# # 공유 폴더
# CO_WORK = "MyDrive/Proj_WT"
# CW_PATH = join(ROOT, CO_WORK)
# DATAS = "DataSets/박정열/"  # EDA 끝낸 데이터
# DATA_PATH = join(CW_PATH, DATAS)
# import numpy as np
# import pandas as pd
# tour_lst = ['덕수궁', '서대문자연사박물관', '서울시립미술관', '선릉정릉',
#             '종묘', '창경궁', '창덕궁', '태릉강릉', '헌릉인릉']
# season_lst = ['봄', '여름', '가을', '겨울']
# def load_csv2(path=None, tour_list=None, season_list=None):
#   dfname_list = []
#   for df in tour_list:
#     for ss in season_list:
#       dfname_list.append(df+"_"+ss)
#   for fname in dfname_list:
#     try:  # 인코딩 에러 피하기 위한 방법
#       globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fname}.csv", encoding='euc-kr')
#       globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]] #혹시 index도 같이 저장되었으면 없애줌
#     except:
#       try:
#         globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fname}.csv", encoding='cp949')
#         globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]]
#       except:
#         globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fname}.csv", encoding='utf-8')
#         globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]]
#   dfname_list = ["DF_"+fname for fname in dfname_list]  #모델 반복문 돌릴 때 필요한 변수(list형태)
#   return dfname_list
# path = DATA_PATH
# dflst = load_csv2(path, tour_lst, season_lst)

#################################################################################################
# 덕수궁 = []
# 서대문자연사박물관 = []

# 관광지 = []
# for i in tour_lst:
#   관광지.append(i)
#   for j in season_lst:
#     data = pd.read_csv(DATA_PATH + i + '_' + j + '.csv')
#     덕수궁.append(data)

# import os
# from google.colab import drive
# # 구글 드라이브 접근
# ROOT = "/content/drive"
# drive.mount(ROOT)
# from os.path import join
# # 공유 폴더
# CO_WORK = "MyDrive/Proj_WT"
# CW_PATH = join(ROOT, CO_WORK)
# DATAS = "DataSets/박정열"  # EDA 끝낸 데이터
# DATA_PATH = join(CW_PATH, DATAS)
# import numpy as np
# import pandas as pd
# def load_csv(path=None):
#   file_list = os.listdir(path)
#   file_list = [fpath for fpath in file_list if '(' not in fpath]
#   dfname_list = []
#   for fpath in file_list:
#     fname = fpath.split('.')[0]
#     dfname_list.append("DF_"+fname) #모델 반복문 돌릴 때 필요한 데이터프레임 변수명 리스트
#     try:  # 인코딩 에러 피하기 위한 방법
#       globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fpath}", encoding='euc-kr')
#       globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]] #혹시 index도 같이 저장되었으면 없애줌
#     except:
#       try:
#         globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fpath}", encoding='cp949')
#         globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]]
#       except:
#         globals()[f"DF_{fname}"] = pd.read_csv(f"{path}/{fpath}", encoding='utf-8')
#         globals()[f"DF_{fname}"] = globals()[f"DF_{fname}"].loc[:, [chk for chk in globals()[f"DF_{fname}"] if "Unnamed" not in chk]]
#   return dfname_list
# path = DATA_PATH
# dflst = load_csv(path)

In [1]:
!pip install vecstack
!pip install catboost
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from vecstack import stacking
from sklearn.ensemble import VotingRegressor

In [2]:
# 전체 데이터를 불러오는 class 만들기
class data_bring:
  def data_bring():
    # 관광지를 dict형태로 저장한다.
    target_map = ['ChangDeokGung', 'ChangGyeongGung', 'DuckSooGung', 'GyeongBokGung', 'HeonLeungInReung', 'JongMyo',
                  'SeoDaeMunNaturalHistoryMuseum', 'SeoulMuseumOfArt', 'SunReungJungReung', 'TaeReungGangNeung', 'TrickEyeMuseum',
                  'SeoDaeMunPrisonHistoryMuseum', 'NamSanGolHanOkVillage', 'NationalMuseumOfKorea']

    # colab path를 찾는다.
    PATH = '/content/drive/MyDrive/Proj_WT/DataSets/박정열/'
    # dict로 담아놓을 공간을 만든다.
    bucket = {}
    # items를 사용하여 각 key와 value를 받는다.
    for i in target_map:
      # 불러올 데이터의 path를 지정한다.
      final_path = PATH + i + '.csv'
      # 데이터를 불러온다.
      data = pd.read_csv(final_path)
      # 관광지에 데이터를 담는다.
      bucket[f'{i}'] = data

    return bucket

# 전처리와 각 모델들을 저장할 class를 만든다.
class modeling:
  # train과 test로 나누기
  def train_test_split(data = None):
    # 2017년 12월 이전을 train
    train = data[data['date'] <= 201712]
    # 2017년 12월 이후를 test
    test = data[data['date'] > 201712]
    # 경복궁 데이터는 최고기온이 없기 때문에 try except구문을 사용하여 빼준다.
    try:
      train.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
      test.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
    except:
      train.drop(['date','평균 기온(°C)'], axis = 1, inplace = True)
      test.drop(['date','평균 기온(°C)'], axis = 1, inplace = True)

    # train을 X_train, y_train으로 나눈다.z
    X_train, y_train = train.drop('target', axis = 1), train['target']
    # test를 X_test, y_test로 나눈다.
    X_test, y_test = test.drop('target', axis = 1), test['target']

    return X_train, X_test, y_train, y_test

  # linear regression 돌리기
  def lr():
    model = LinearRegression()
    
    return model
  # xgboost 돌리기
  def xgb():
    model = XGBRegressor(random_state = 42,tree_method = 'gpu_hist')

    return model
  # lightgbm
  def lgb():
    model = LGBMRegressor(random_state = 42)

    return model
  # catboost
  def cat():
    model = CatBoostRegressor(random_state = 42,task_type = 'GPU')
    return model
  # ridge
  def ridge():
    model = Ridge(random_state = 42)
    
    return model
  
  # lasso
  def lasso():
    model = Lasso()

    return model

  # adaboost
  def ada():
    model = AdaBoostRegressor(random_state = 42)

    return model

  # randomforest
  def rfg():
    model = RandomForestRegressor(random_state = 42)

    return model

  # rmse가 가장 낮은 것을 데이터 프레임으로 rmse함수를 만든다.
  def rmse(y_test, y_pred):
    RMSE = mean_squared_error(y_test, y_pred)**0.5

    return RMSE

# 모델 전부 한번씩 돌려보기
class class_all_model:
  # 각 모델을 돌리기 위해서 dict로 쌓아준다.
  def models():
    models = {'lr':modeling.lr(), 'lgb':modeling.lgb(), 'xgb':modeling.xgb(), 'lasso':modeling.lasso(),'ridge':modeling.ridge(),'ada':modeling.ada(), 'rfg':modeling.rfg()}
    return models
  
  # 모델을 하나씩 돌려서 keys = 모델명, value = 모델의 rmse를 쌓아준다.
  def run(data = None):
    models = class_all_model.models()
    # 모델별로 rmse를 넣는다.
    final_rmse = {}
    # 모델별로 score를 넣는다.
    model_score = {}
    # 모델별로 
    models_store = {}
    X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
    # dict로 쌓아놓은 데이터를 전부 가져온다.
    for i, j in models.items():
      # 각 모델의 y_pred를 구한다.
      model = j
      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)
      
      # 전체 모델을 돌려서 각 모델별로 rmse, score,model을 저장한다.
      final_rmse[i] = modeling.rmse(y_test, y_pred)
      model_score[i] = model.score(X_test, y_test)
      models_store[i] = model

    return final_rmse, model_score, models_store

# dict로 쌓여있는 모델을 돌린다.
class final_model:
  def run(data = None):
    # 위의 class에서 모델을 돌려서 rmse, score, model을 불러온다.
    final_rmse , model_score, model = class_all_model.run(data = data)
    
    # model_score의 key = 모델 , value = 모델의 score를 받는다.
    for i, j in model_score.items():
      # score가 가장 높은 모델을 찾아낸다.
      if j == max(model_score.values()):
        # score가 가장 높은 모델을 dataframe의 형태로 만들기 위한 작업을 한다.
        model = model[i]
        final_rmse = final_rmse[i]
        model_score = model_score[i]
        break

    # 가장 score가 높은 모델의 모델, rmse, score를 데이터 프레임 형태로 저장한다.
    score = pd.DataFrame([[model, final_rmse, model_score]], columns = ['model','final_rmse','model_score'], index = [i])

    return score

# 각 관광지의 데이터를 하나의 데이터 프레임에 보기 위해서 class 를 생성한다.
class data_concat:
  def final_run():
    # 관광지별 데이터를 전부 불러온다.
    final_dict = data_bring.data_bring()
    # 각 관광지별 rmse, model, score를 저장할 데이터 프레임을 만든다.
    final_dataframe = pd.DataFrame([])
    # 각 관광지별 데이터를 가져온다.
    for i, j in final_dict.items():
      # 각 관광지별 데이터를 위에 선언된 class로 모델을 돌린다.
      dataframe = final_model.run(data = j)
      # 관광지라는 변수를 지정해준다.
      # 왜 이렇게 하는가? index에 바로 안들어가기 때문이다.
      dataframe['TourSpot'] = i
      # 관광지를 concat으로 쌓는다.
      final_dataframe =  pd.concat([final_dataframe,dataframe], axis = 0)
    # 최종 데이터 프레임이 만들어지면 관광지를 index로 변경해준다.
    final_dataframe.index = final_dataframe['TourSpot']
    # 관광지라는 컬럼을 제거한다.
    final_dataframe.drop('TourSpot', axis = 1, inplace = True)

    return final_dataframe

In [ ]:
# # 데이터를 dict형태로 쌓아서 각 관광지를 key 관광지의 계절을 value로 받게 만들었다.
# class data_bring:
#   def data_bring():
#     target_map = {'ChangDeokGung' : '창덕궁', 'ChangGyeongGung' : '창경궁', 'DuckSooGung' : '덕수궁', 'GyeongBokGung' : '경복궁', 'HeonLeungInReung' : '헌릉인릉', 'JongMyo' : '종묘', 'SeoDaeMunNaturalHistoryMuseum' : '서대문자연사박물관', 'SeoulMuseumOfArt' : '서울시립미술관', 'SunReungJungReung' : '선릉정릉', 'TaeReungGangNeung' : '태릉강릉', 'TrickEyeMuseum' : '트릭아이미술관', 'SeoDaeMunPrisonHistoryMuseum' : '서대문형무소역사관', 'NamSanGolHanOkVillage' : '남산골한옥마을', 'NationalMuseumOfKorea' : '국립중앙박물관'}
#     # season_map = ['','Spring','Summer','Fall','Winter']
#     PATH = '/content/drive/MyDrive/Proj_WT/DataSets/박정열/'
#     담아놓기 = {}
#     for i, j in target_map.items():
#       final_path = PATH + i + '.csv'
#       data = pd.read_csv(final_path)
#       담아놓기[f'{i}'] = data

#     return 담아놓기

# class modeling:
#   # train과 test로 나누기
#   # 전처리에서 했어야할 drop을 여기서 해주네... 뭐 그냥 그렇다구요
#   def train_test_split(data = None):
#     train = data[data['date'] <= 201712]
#     test = data[data['date'] > 201712]

#     try:
#       train.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
#       test.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
#     except:
#       train.drop(['date','평균 기온(°C)'], axis = 1, inplace = True)
#       test.drop(['date','평균 기온(°C)'], axis = 1, inplace = True)

#     X_train, y_train = train.drop('target', axis = 1), train['target']
#     X_test, y_test = test.drop('target', axis = 1), test['target']

#     return X_train, X_test, y_train, y_test

#   # linear regression 돌리기
#   def lr():
#     model = LinearRegression()
    
#     return model

#   # xgboost 돌리기
#   def xgb():
#     model = XGBRegressor(random_state = 42,tree_method = 'gpu_hist')

#     return model

#   # lightgbm
#   def lgb():
#     model = LGBMRegressor(random_state = 42)

#     return model

#   # catboost
#   def cat():
#     model = CatBoostRegressor(random_state = 42,task_type = 'GPU')

#     return model

#   # ridge
#   def ridge():
#     model = Ridge(random_state = 42)
    
#     return model
  
#   # lasso
#   def lasso():
#     model = Lasso()

#     return model

#   # adaboost
#   def ada():
#     model = AdaBoostRegressor(random_state = 42)

#     return model

#   # randomforest
#   def rfg():
#     model = RandomForestRegressor(random_state = 42)

#     return model

#   # rmse가 가장 낮은 것을 데이터 프레임으로 rmse함수를 만든다.
#   def rmse(y_test, y_pred):
#     RMSE = mean_squared_error(y_test, y_pred)**0.5

#     return RMSE

# # 모델 전부 한번씩 돌려보기
# class class_all_model:
#   # 각 모델을 돌리기 위해서 dict로 쌓아준다.
#   def models():
#     models = {'lr':modeling.lr(), 'lgb':modeling.lgb(), 'xgb':modeling.xgb(), 'lasso':modeling.lasso(),'ridge':modeling.ridge(),'ada':modeling.ada(), 'rfg':modeling.rfg()}
#     return models
  
#   # 모델을 하나씩 돌려서 keys = 모델명, value = 모델의 rmse를 쌓아준다.
#   def run(data = None):
#     models = class_all_model.models()
#     final_models = {}
#     model_score = {}
#     model_binning = {}
#     X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
#     # dict로 쌓아놓은 데이터를 전부 가져온다.
#     for i, j in models.items():
#       model = j
#       model.fit(X_train, y_train)
#       y_pred = model.predict(X_test)
      
#       # 전체 모델을 돌려서 rmse가 가장 낮은 데이터를 찾아온다.
#       final_models[i] = modeling.rmse(y_test, y_pred)
#       model_score[i] = model.score(X_test, y_test)
#       model_binning[i] = model

#     return final_models, model_score, model_binning

# # stacking ensemble
# # class class_stack:
# #   # stacking에 사용할 모델들을 stacking에 바로 넣을 수 있게 만들어준다.
# #   def model_stack():
# #       models = [modeling.lgb(), modeling.cat(), modeling.lr(), modeling.xgb(), modeling.lasso(), modeling.ada(), modeling.rfg(),modeling.ridge()]
# #       return models
# #   # stacking모듈을 사용
# #   def stack(data = None):
# #     X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)

# #     S_train, S_test = stacking(class_stack.model_stack(), X_train, y_train, X_test, regression = True, metric = ['acc'], n_folds = 3)
# #     model = modeling.lgb()
# #     model.fit(S_train, y_train)
# #     y_pred = model.predict(S_test)
# #     model_score = model.score(S_test, y_test)

#     # return modeling.rmse(y_test, y_pred), model_score, model
# # voting ensemble
# # class class_voting:
# #   # 모델들을 voting에 맞게 모아준다.
# #   def collect_model():
# #     models= [('xgb',modeling.xgb()),('lgb',modeling.lgb()),('rfg',modeling.rfg()),('ada',modeling.ada()),('lr',modeling.lr()),('ridge',modeling.ridge()),('lasso',modeling.lasso()),('cat',modeling.cat())]
    
# #     return models

#   # VotingRegressor를 돌려준다.
#   # def voting(data = None):
#   #   X_train, X_test, y_train, y_test = modeling.train_test_split(data = data)
#   #   model = VotingRegressor(class_voting.collect_model(), )
#   #   model.fit(X_train, y_train)
#   #   y_pred = model.predict(X_test)
#   #   model_score = model.score(X_test, y_test)
#   #   return modeling.rmse(y_test, y_pred), model_score, model

# # class로 짠 모델들을 전부 돌려준다.
# class final_model:
#   def run(data = None):
#     final_rmse , model_score, model = class_all_model.run(data = data)
#     # final_rmse['stack'], model_score['stack'], model['stack'] = class_stack.stack(data = data)
#     # final_rmse['voting'], model_score['voting'], model['voting'] = class_voting.voting(data = data)

#     for i, j in model_score.items():
#       if j == max(model_score.values()):
#         model = model[i]
#         final_rmse = final_rmse[i]
#         model_score = model_score[i]
#         break

#     score = pd.DataFrame([[model, final_rmse, model_score]], columns = ['model','final_rmse','model_score'], index = [i])

#     return score

# # 계절별로 데이터 프레임 만들기
# class data_concat:
#   def data_concat(data = [None]):
#     final_data = pd.DataFrame([], columns = ['model','final_rmse','model_score'])
#     a = final_model.run(data = data)
#     final_data = pd.concat([final_data, a], axis = 0)

#     return final_data

#   def final_run():
#     final_dict = data_bring.data_bring()
#     final_dataframe = pd.DataFrame([])
#     for i, j in final_dict.items():
#       dataframe = data_concat.data_concat(data = j)
#       dataframe['관광지'] = i
#       final_dataframe =  pd.concat([final_dataframe,dataframe], axis = 0)
#     return final_dataframe

#   # def 
#     print('------ 끝 --------------------------------------------------')

In [ ]:
# 학습된 모델 저장하기
import pickle
import joblib

final_Df = data_concat.final_run()
for i in final_Df.index:
  model = final_Df.loc[i, 'model']
  save_model = pickle.dumps(model)
  joblib,dump(save_model, f'/content/drive/MyDrive/Proj_WT/ModelScore/{i}의 모델.pkl')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


[02:31:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18371229387.231285, tolerance: 10809040.78287024
  positive)


[02:31:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4954022074.981812, tolerance: 25882021.425780952
  positive)


...

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2366882890.6205063, tolerance: 20156538.27507024
  positive)


[02:31:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3049385067.4040833, tolerance: 57089850.47728096
  positive)


[02:31:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


NameError: ignored

In [ ]:
print(final_Df)

In [ ]:
# class를 이용해서 모든 모델을 돌려준다.
######## base line ############################
# final_rmse, model_score, model = final_model.run(data = data)
# print(final_rmse, model_score, model)

In [ ]:
# 더미화, 

In [ ]:
# class visualizer_class:

In [ ]:
# import statsmodels.api as sm

# train = data[data['date'] <= 201712]
# test = data[data['date'] > 201712]




# X_train, y_train = train.drop('target', axis = 1), train['target']
# X_test, y_test = test.drop('target', axis = 1), test['target']


# model = sm.OLS(y_train, X_train)
# result = model.fit()
# result.summary()

In [ ]:
# y_test

In [ ]:
# 데이터를 dict형태로 쌓아서 각 관광지를 key 관광지의 계절을 value로 받게 만들었다.
# import pandas as pd
# target_map = {'ChangDeokGung' : '창덕궁', 'ChangGyeongGung' : '창경궁', 'DuckSooGung' : '덕수궁', 'GyeongBokGung' : '경복궁', 'HeonLeungInReung' : '헌릉인릉', 'JongMyo' : '종묘', 'SeoDaeMunNaturalHistoryMuseum' : '서대문자연사박물관', 'SeoulMuseumOfArt' : '서울시립미술관', 'SunReungJungReung' : '선릉정릉', 'TaeReungGangNeung' : '태릉강릉', 'TrickEyeMuseum' : '트릭아이미술관', 'SeoDaeMunPrisonHistoryMuseum' : '서대문형무소역사관', 'NamSanGolHanOkVillage' : '남산골한옥마을', 'NationalMuseumOfKorea' : '국립중앙박물관'}
# season_map = ['','Spring','Summer','Fall','Winter']

# PATH = '/content/drive/MyDrive/Proj_WT/DataSets/박정열/'
# 담아놓기 = {}
# for i, j in target_map.items():
#   데이터 = []
#   for k in season_map:
#     try:
#       final_path = PATH + i + '_' + k + '.csv'
#       data = pd.read_csv(final_path)
#       데이터.append(data)
#     except:
#       final_path = PATH + i + '.csv'
#       data = pd.read_csv(final_path)
#       데이터.append(data)
#   담아놓기[j] = 데이터

In [ ]:
# final_data = data_bring.data_bring()
# len(final_data['창덕궁'])

In [ ]:
# final_data.keys()

# 딥러닝으로 비교해보기

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Proj_WT/DataSets/박정열/DuckSooGung.csv')

In [5]:
data.head()

,date,month,최고 기온(°C),최소 상대습도(%),평균 기온(°C),평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조 시간(hr),0.5m 지중온도(°C),최대 풍속(m/s),합계 일사량(MJ/m2),target,season,tp_1_0,tp_1_1,tp_1_2,tp_1_3,tp_2_0,tp_2_1,tp_2_2,tp_2_3
0,201101,1,-3.406452,35.129032,-7.183871,53.829032,1.112500,2.796774,7.048387,1.541935,5.593548,9.951290,92135,4.0,1,0,0,0,1,0,0,0
1,201102,2,5.864286,32.000000,1.221429,55.228571,9.700000,2.553571,5.950000,1.025000,5.239286,10.827857,132920,4.0,1,0,0,0,1,0,0,0
2,201103,3,8.345161,25.258065,3.616129,51.112903,2.085714,3.406452,7.748387,5.158065,6.877419,16.342258,64869,1.0,0,1,0,0,0,1,0,0
3,201104,4,15.596667,28.666667,10.720000,54.240000,10.009091,3.243333,6.736667,10.736667,6.650000,16.832333,85548,1.0,0,1,0,0,0,1,0,0
4,201105,5,22.983871,32.161290,17.925806,56.416129,4.107692,2.832258,5.819355,16.303226,6.235484,17.091935,103100,1.0,0,0,1,0,0,0,1,0


In [6]:
# class Generator(nn.Module):
#   def __init__(self, in_channels = 3, z_dim = 100):


In [7]:
train = data[data['date'] <= 201712]
test = data[data['date'] > 201712]
train.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
test.drop(['date','최고 기온(°C)','평균 기온(°C)'], axis = 1, inplace = True)
X_train, y_train = train.drop('target', axis = 1), train['target']
X_test, y_test = test.drop('target', axis = 1), test['target']

from sklearn.preprocessing import RobustScaler
# robust 스케일러를 사용하여 데이터를 처리 해준다.
transform = RobustScaler().fit(X_train)
X_train = transform.transform(X_train)
transform = RobustScaler().fit(X_test)
X_test = transform.transform(X_test)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten, BatchNormalization
import tensorflow as tf

def rmse(true, pred):
  RMSE = mean_squared_error(true, pred)** 0.5
  return RMSE

# shape 맞춰주기
X_train_t = X_train.reshape(X_train.shape[0], 18,1 )
X_test_t = X_test.reshape(X_test.shape[0], 18,1 )
print(X_train_t.shape, X_test_t.shape, y_train.shape, y_test.shape)
y_train = y_train.values
y_test = y_test.values

# LSTM 사용하기
model = Sequential()
model.add(LSTM(1024, activation = 'relu', input_shape = (X_train_t.shape[1], X_train_t.shape[2], ), return_sequences = True))
model.add(Dropout(0.1))
model.add(LSTM(512, activation = 'relu', return_sequences = True))
model.add(Dropout(0.1))
model.add(LSTM(256, activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))
BatchNormalization()
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))

model.add(Flatten())
model.add(Dense(1))

model.compile(loss = 'mae', optimizer = 'adam', metrics = ['mean_squared_error'])
print(model.summary())

model.fit(X_train_t, y_train, epochs = 50, batch_size = 1, validation_data = (X_test_t, y_test))
y_pred = model.predict(X_test_t)

(84, 18, 1) (24, 18, 1) (84,) (24,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 18, 1024)          4202496   
_________________________________________________________________
dropout (Dropout)            (None, 18, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 18, 512)           3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 18, 256)           787456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)       

In [9]:
y_pred = model.predict(X_test_t)

In [10]:
modeling.rmse(y_test, y_pred)

78595.27884098377

In [ ]:
y_test

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Proj_WT/DataSets/박정열/DuckSooGung.csv')
minmax = pd.read_csv('/content/drive/MyDrive/sd솔데스크/팀 프로젝트/DuckSooGung_minmax.csv')
robust = pd.read_csv('/content/drive/MyDrive/sd솔데스크/팀 프로젝트/DuckSooGung_robust.csv')
standard = pd.read_csv('/content/drive/MyDrive/sd솔데스크/팀 프로젝트/DuckSooGung_standard.csv')

In [ ]:
final_model.run(data)

In [ ]:
final_model.run(minmax)

In [ ]:
final_model.run(robust)

In [ ]:
final_model.run(standard)